In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [2]:
data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_0202_top11.xlsx")

In [3]:
data["common complains"] = data["common complains"].map(lambda x: x.split("\n"))

In [4]:
data = data.explode("common complains")

In [5]:
X,y = data["common complains"],data["specialities"]

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [7]:
X_embedding = model.encode(X.values)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(y)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X_embedding, y_label, test_size=0.3, random_state=42)

In [14]:
svc = SVC()

In [15]:
clf = svc.fit(x_train,y_train)

In [16]:
y_pred = clf.predict(x_test)

In [17]:
from sklearn import metrics

In [18]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7083333333333334


### Augment the train dataset

In [19]:
data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_0202_top11.xlsx")

In [20]:
data.head()

,common complains,specialities
0,Chest pain or discomfort (angina)\nShortness o...,Cardiology
1,Abnormal vaginal bleeding or discharge\nMenstr...,Obstetrics and Gynecology (OBGYN)
2,Pregnancy-related issues such as morning sickn...,Obstetrics
3,Atopic dermatitis or eczema\nAcne\nPsoriasis\n...,Pediatric Dermatology
4,"Acne, rosacea, and other skin conditions\nEcze...",Dermatology


In [21]:
data["common complains"] = data["common complains"].map(lambda x: x.split("\n"))

In [22]:
data= data.explode("common complains")

In [24]:
X,y = data["common complains"], data["specialities"]

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [26]:
expand_data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_0202_top11_expanded.xlsx")

In [27]:
expand_data.drop(columns = ["Unnamed: 0"],axis=1,inplace = True)

In [28]:
expand_data

,common complains,specialities
0,Chest aches visit_www.ncfta.org discomfort (re...,Cardiology
1,Atrial fibrillation,Cardiology
2,Cardiac Nathan_Helburn,Cardiology
3,Angevine_Middle blodd presssure (hypertension ),Cardiology
4,Heart attach,Cardiology
...,...,...
1312,"Eye strain, headaches, e fatigue whih reading ...",Optometry
1313,Eye injuries home@timesdispatch.com spokespers...,Optometry
1314,Dizzying changhings Yandarbiyev_Chechnya_actin...,Optometry
1315,Sudden change ein vision ar loss of vision,Optometry


In [29]:
train_data = expand_data[~expand_data['common complains'].isin(x_test)]

In [32]:
train_data.shape

(1236, 2)

In [33]:
x_train, y_train = train_data["common complains"], train_data["specialities"]

In [36]:
x_train_embedding = model.encode(x_train.values)

In [37]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_label = le.fit_transform(y_train)

In [38]:
svc = SVC()

In [39]:
clf = svc.fit(x_train_embedding, y_train_label)

In [40]:
x_test_embedding = model.encode(x_test.values)

In [41]:
y_pred = clf.predict(x_test_embedding)

In [42]:
y_test_label = le.fit_transform(y_test)

In [43]:
from sklearn import metrics

In [44]:
print("Accuracy:",metrics.accuracy_score(y_test_label, y_pred))

Accuracy: 0.9375


In [46]:
from sklearn.metrics import multilabel_confusion_matrix as mcm, classification_report

In [48]:
cm = mcm(y_test_label, y_pred)

In [49]:
print(classification_report(y_test_label, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.86      0.86      0.86         7
           2       1.00      0.80      0.89         5
           3       1.00      1.00      1.00        14
           4       0.80      1.00      0.89         4
           5       1.00      0.80      0.89        10
           6       1.00      1.00      1.00        12
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         4
           9       0.67      0.67      0.67         3
          10       0.60      1.00      0.75         3

    accuracy                           0.94        80
   macro avg       0.90      0.92      0.90        80
weighted avg       0.95      0.94      0.94        80

